Content

1. [Data Cleaning and Preparation](#data_prep)
2. [Use cases](#users)
    - [Use Case 1](#use1)
    - [Use Case 2](#use2)    
    - [Use Case 3](#use3)    

<a id='data_prep'></a><h1>1. Data Cleaning and Preparation</h1>

In [1]:
import pandas as pd
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen

In [2]:
excel = pd.ExcelFile('SocMed_Geographic.xlsx')
df = pd.read_excel(excel, 'Sheet1')

In [3]:
def rename_col(df):
    df.fillna(0, inplace=True)
    df_col = {}
    
    for i in range(len(df.columns)): 
        existing_col_name = df.columns[i]
        if i != 0: 
            country = df.iloc[0][i]
            df_col[existing_col_name] = country.strip()
        else: 
            df_col[existing_col_name] = existing_col_name
    
    df.rename(columns=df_col, inplace=True)
    return df.drop(0).reset_index(drop=True)

In [4]:
def remove_brackets(df):
    for i in range(len(df)): 
        for j in range(1, len(df.columns)): 
            
            col = df.iloc[i][j]
            if type(col) == str:
                try:
                    bracket_pos = col.index('(')
                    value = col[:bracket_pos-1]
                    df.iloc[i, j] = value
                except: 
                    pass
                
    return df

In [5]:
def clean_df1():
    for i in range(len(df)): 
        for j in range(1, len(df.columns)): 

            col = df.iloc[i][j]
            if type(col) == str and ' ' in col: 
                value = col.replace(" ", "")
            elif type(col) == str and ',' in col:
                value = col.replace(",", "")
            else: 
                value = col

            try: 
                df.iloc[i, j] = int(value)
            except: 
                df.iloc[i, j] = int(value.replace('\u202c', '')) 

In [6]:
df = rename_col(df)

df = remove_brackets(df)

clean_df1()

In [7]:
url = 'https://en.m.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')
tables = soup.find_all('table')

In [8]:
population_dic = {}

for table in tables:
    rows = table.find_all('tr')
    
    for i in range(len(rows)):
        if i > 1: 
            row = rows[i]
            cells = row.find_all('td')
            
            country = cells[0].text.strip()
            population = cells[2].text.strip()
            population = population.replace(",", "")
            population_dic[country] = int(population)

<a id='users'></a><h1>2. Use cases</h1>

<a id='use1'></a><h2>2a. Use case 1</h2>

#1: When I choose a social media network, I am able to get a view the number of users in each country.

In [9]:
# do not change the code below
def filter_social(social, df=df): 
    social_df = df[df['Social Media Platform'] == social]
    social_df = social_df.transpose()
    social_df = social_df.rename(columns={'':'Country', 0:social})
    social_df = social_df.drop('Social Media Platform')
    return social_df

In [10]:
filter_social('Bilibili')

,Bilibili
United States,0
Indonesia,0
Singapore,0
China,27400000
India,0
Vietnam,0
Philippines,0
Bangladesh,0


<a id='use2'></a><h2>2b. Use case 2</h2>

#2: When I choose a country, I am able to see all the relevant social networks its people use ranked in descending order.

In [11]:
# do not change the code below
def filter_country(country, df=df): 
    sort_country_df = df.loc[: , ['Social Media Platform', country]]
    return sort_country_df.sort_values(by=country, ascending=False)

In [12]:
filter_country('United States')

,Social Media Platform,United States
9,Youtube,192000000
1,Facebook,190000000
3,Linkedln,160000000
2,Instagram,107000000
6,Twitter,55700000
5,Tik Tok,45400000
0,Bilibili,0
4,Tencent QQ,0
7,WeChat,0
8,Weibo,0


<a id='use3'></a><h2>2c. Use case 3</h2>

#3: I can view a comparison of the percentage of users based on the population for each social media network by the population of the country. (You will have to scrape the population data and provide the source(s).)

In [13]:
# do not edit the code below
def get_percentage(country):
    filtered_df = filter_country(country)

    for row, col in filtered_df.iterrows(): 
        users = col[1]
        population = population_dic[country]
        percent = users / population * 100
        percentage = f"{percent:.2f}%"
        col[1] = percentage
        
    return filtered_df

In [14]:
get_percentage('United States')

,Social Media Platform,United States
9,Youtube,57.70%
1,Facebook,57.10%
3,Linkedln,48.08%
2,Instagram,32.16%
6,Twitter,16.74%
5,Tik Tok,13.64%
0,Bilibili,0.00%
4,Tencent QQ,0.00%
7,WeChat,0.00%
8,Weibo,0.00%
